# 1.2 Querying a Public Model (GPT-2)
**Protecting Individual's privacy**

## Dependencies

Make you sure you have [pytorch-transformers](https://github.com/huggingface/pytorch-transformers) installed. PyTorch-Transformers can be installed by pip as follows:

In [1]:
# !pip install pytorch-transformers

In [2]:
import torch as th
import grid as gr
from grid import syft as sy

# GPT-2 model
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

In [3]:
hook = sy.TorchHook(th)

## Connect to a GridWorker

In [4]:
app_company = gr.WebsocketGridClient(hook, "http://localhost:3000/", id="app_company")
app_company.connect()

## Model and Data Setup

In [5]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2',torchscript=True)

In [6]:
# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = th.tensor([indexed_tokens])

## Querying a Public Model

We can get a copy of the model by calling `worker.download_model(model_id)`

In [7]:
model_copy = app_company.download_model("GPT-2-public")

Traceback (most recent call last):
  File "/home/marianne/Grid/grid/client.py", line 160, in download
    return sy.hook.local_worker.fetch_plan(model_id, self, copy=True)
  File "/home/marianne/PySyft/syft/workers/base.py", line 805, in fetch_plan
    plan.replace_worker_ids(location.id, self.id)
AttributeError: 'NoneType' object has no attribute 'replace_worker_ids'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/marianne/.local/lib/python3.6/site-packages/urllib3/connectionpool.py", line 399, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/home/marianne/.local/lib/python3.6/site-packages/urllib3/util/response.py", line 72, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [StartBoundaryNotFoundDefect(), MultipartInvariantViolationDefect()], unparsed data: ''


And run it locally!

In [8]:
%%time
response = model_copy(tokens_tensor)

predictions = th.tensor(response[0])
predicted_index = th.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print("Predicted text: ", predicted_text)

Predicted text:   Who was Jim Henson? Jim Henson was a great
CPU times: user 404 ms, sys: 44.1 ms, total: 448 ms
Wall time: 161 ms


/home/marianne/PySyft/syft/frameworks/torch/hook/hook.py:483: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)
